In [16]:
import pandas as pd
import os

# Load data

In [17]:
script_dir = os.path.dirname("__file__")  # the cwd relative path of the script file


# Bhopal dataset 
bhopal_ds_name = "data/Bhopal_CompleteData_Final_Modified.csv"
rel_to_bhopal_ds = os.path.join(script_dir, bhopal_ds_name)
df_bhopal = pd.read_csv(rel_to_bhopal_ds)
df_bhopal.index = pd.to_datetime(df_bhopal.date)


# Algiers dataset
algeria_ds_name = "data/GLDAS_GFS_population_Algeria_2015_2023_final.csv"
rel_to_algeria_ds = os.path.join(script_dir, algeria_ds_name)
df_algeria = pd.read_csv(rel_to_algeria_ds)
df_algeria.index = pd.to_datetime(df_algeria.date)


In [18]:
# Create target column for Algeria

# Set constants
water_supply= 490
# Set the total lake and reservoir area to lake_reservoir_area
total_dam_area = 75.63
# A day in second
day_in_sec = 86400
# Water demand'
daily_water_demand_lcd = 200

# Calculate daily water volume in liter
precipitation_mld = total_dam_area * df_algeria['precipitation_sum (mm)']
evapotranspiration_mld = total_dam_area * 10**6 * (df_algeria['Evap_tavg'] * day_in_sec * 10**(-6))
runoff_mld = total_dam_area * 10**6 * (df_algeria['Qs_tavg'] * day_in_sec * 10**(-6))
delta = precipitation_mld - evapotranspiration_mld - runoff_mld
df_algeria['daily_water_volume'] = water_supply + delta

In [19]:
# Create target column for Bhopal

# The area of the Upper lake in km^2
Upper_lake_area = 36
# Fixed storage (S) value (in MLD) as explained from the above calculations
Storage = 398.05

# Precipitation in MLD
df_bhopal['Precipitation in MLD'] = df_bhopal['precipitation_sum']*Upper_lake_area

# Evapotranspiration in MLD
df_bhopal['Evapotranspiration in MLD'] = df_bhopal['Evap_tavg']*Upper_lake_area*86.4*10**(3)

# Surface water runoff in MLD
df_bhopal['Surface Water Runoff in MLD'] = df_bhopal['Qs_tavg']*Upper_lake_area*86.4*10**(3)

# Change in storage (del S) in MLD
df_bhopal['del_S'] = df_bhopal['Precipitation in MLD']-df_bhopal['Evapotranspiration in MLD']-df_bhopal['Surface Water Runoff in MLD']

# Daily Quantity of water supplied in MLD
df_bhopal['daily_water_volume'] = Storage + df_bhopal['del_S']

In [20]:
# Aggregate daily data to monthly level, Bhopal
TARGET = 'daily_water_volume'
df_bhopal['year'] = df_bhopal.index.year
df_bhopal['month'] = df_bhopal.index.month
df_bhopal_month_agg = df_bhopal.groupby(['year', 'month'])[TARGET].mean().reset_index()
df_bhopal_month_agg['date_ym'] = df_bhopal_month_agg['year'].astype(str) + '-' + df_bhopal_month_agg['month'].astype(str)
df_bhopal_month_agg.index = pd.to_datetime(df_bhopal_month_agg['date_ym'])
df_bhopal_month_agg.resample('M').last()
df_bhopal_month_agg=df_bhopal_month_agg[[TARGET]]
df_bhopal_month_agg.head()
df_bhopal_month_agg.to_csv('data/bhopal_month_agg_final.csv', index_label=False)

In [21]:
# Aggregate daily data to monthly level, Algeria
df_algeria['year'] = df_algeria.index.year
df_algeria['month'] = df_algeria.index.month
df_algeria_month_agg = df_algeria.groupby(['year', 'month'])[TARGET].median().reset_index()
df_algeria_month_agg['date_ym'] = df_algeria_month_agg['year'].astype(str) + '-' + df_algeria_month_agg['month'].astype(str)
df_algeria_month_agg.index = pd.to_datetime(df_algeria_month_agg['date_ym'])
df_algeria_month_agg.resample('M').last()
df_algeria_month_agg=df_algeria_month_agg[[TARGET]]
df_algeria_month_agg.head()
df_algeria_month_agg.to_csv('data/algeria_month_agg_final.csv', index_label=False)